In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import gensim
from gensim import corpora, models
from pprint import pprint

возьмём для анализа данные чеспионата мира по футолу 2018 года

In [ ]:
import pandas as pd
fifa = pd.read_csv("../input/world-cup-2018-tweets/FIFA.csv")

In [ ]:
fifa.head(5)

In [ ]:
like_sort = fifa.sort_values(by='Likes', ascending=False)
like_sort.head(5)

выберем 1000 наиболее популярных комментариев

In [ ]:
data = like_sort.head(1000)

In [ ]:
document = [data['Tweet'].values[i] for i in range(0,1000)]

In [ ]:
document = [str(x) for x in document]

Токенизация полученных документов

In [ ]:
text = [[text for text in x.split()] for x in document]

In [ ]:
diction = corpora.Dictionary(text)

Получение информации о словаре

In [ ]:
print(diction)

Мы успешно создали объект Dictionary. 

Gensim будет использовать этот словарь для создания корпуса, в котором слова в документах заменяются соответствующим идентификатором, предоставленным этим словарем. Все объекты корпуса содержит id слова и его частоту в каждом документе.

После создания объекта Dictionary, все, что вам нужно сделать, это передать токенизированный список слов в **Dictionary.doc2bow()**

In [ ]:
corps = [diction.doc2bow(doc, allow_update=True) for doc in text]

Сохраним словарь и построенный корпус

In [ ]:
# Save the Dict and Corpus
diction.save('fifa2018.dict')  # save dict to disk
corpora.MmCorpus.serialize('fifa2018corpus.mm', corps)  # save corpus to disk

Построим матрицу TFIDF

In [ ]:
tfidf = models.TfidfModel(corps, smartirs='ntc')
for doc in tfidf[corps]:
    print([[diction[id], np.around(freq, decimals=3)] for id, freq in doc])

N-грамма — последовательность из n элементов. С семантической точки зрения, это может быть последовательность звуков, слогов, слов или букв. На практике чаще встречается N-грамма как ряд слов, устойчивые словосочетания называют коллокацией. Последовательность из двух последовательных элементов часто называют биграмма, последовательность из трёх элементов называется триграмма. Не менее четырёх и выше элементов обозначаются как N-грамма, N заменяется на количество последовательных элементов.

Построим биграммы по ранее созданном списку твитов, хранимому в переменной document

In [ ]:
bigram = gensim.models.phrases.Phrases(text, min_count=3, threshold=1)
print(bigram[text[0]])

Полученный результат не выявил биграмм для первого твита, попробуем прросмотреть первые 10

In [ ]:
for i in range(0,10):
    print(bigram[text[i]])

Как видим, биграммы были успешно построены и среди них можно заметить как довольно предсказуемые, такие как сочетание имя_фамилия поупулярных тренеров и игроков, так и довольно неожиданные, например "face_Russia"

Можно предположить, что в данном конкретном случае данное выражение не относится к чьему-либо конкретному лицу, а относится к облику страны, её условному статусу.
Из чистого любопытства получим все коментарии, содержащие вхождение данной биграммы

In [ ]:
counter = 0

for i in document:
    counter += 1 
    if 'face Russia' in i:
        print(str(counter) + ' - '+i)        

Отлично, у нас есть 4 твита с искомой биграммой и исходное предположение.
Беглый просмотр твитов говорит нам, что мы ошиблись, предполагая, что твиты говорят об облике в глазах мирового сообщетсва.

Смылс всех 4 коментариев сводится к прекраснойй русской фразе "встретиться лицом к лицу"

Давайте попробуем построить триграммы

In [ ]:
# Build the trigram models
trigram = gensim.models.phrases.Phrases(bigram[text], threshold=10)
# Construct trigram
print(trigram[bigram[text[0]]])

Логично предположить, что найти триграммы будет сложнее, нежели устойчивые выражения из 2 слов
Однако, если мы внимательно смотрели 4 твита с биграммой "face Russia", мы могли заметить потенциальную триграмму "will face Russia"

Но, есть способ проще и быстрее, а именно регулярные выражения. Выведем все уникальные триграммы, для исследуемого набора данных.

In [ ]:
from re import fullmatch

In [ ]:
lst = []

for x in range(0, 1000):
    for i in trigram[bigram[text[x]]]:
        if fullmatch('\w+_\w+_\w+', i) != None:
            if i not in lst:
                lst.append(i)
                print(i)
        
        

как мы видим, некоторые триграммы составлены из устойчивых конструкций английского языка, таких как 'in the' или же 'at the', в то же время, предположение о потенциальной тиграмме не подтвердилось "will face Russia"

Попытаемся обучить модель word2vec на основе полученных данных

In [ ]:
from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count

In [ ]:
w2v_model = Word2Vec(text, min_count = 0, workers=cpu_count())

Учитывая происхождение датасета в качестве примера выведем полученные вектора для слова 'football'

In [ ]:
w2v_model['football']

Теперь получим слова наиболее близкие по смыслу к слову 'football'

In [ ]:
w2v_model.most_similar('football')

Как видим, результат получился крайне противоречивым. Это можно объяснить небольшим количеством данных, взятых нами для обработки - всего 75 документов, длиной равной или менее 280 символов.

Попробуем использовать 100000 записей и сравним результаты.
Для этого нам необходимо будет обновить нашу модель.

In [ ]:
data_new = like_sort.head(100000)
document_new = [str(data_new['Tweet'].values[i]) for i in range(0,100000)]
text_new = [[text for text in x.split()] for x in document_new]

In [ ]:
w2v_model.build_vocab(text_new, update=True)
w2v_model.train(text_new, total_examples=w2v_model.corpus_count, epochs=w2v_model.iter)

In [ ]:
w2v_model.most_similar('football')

Как можем увидеть, результаты значительно улучшились и теперь мы знаем, что football почти = soccer и это традиционный английский спорт

Сохраним нашу модель

In [ ]:
w2v_model.save('fifa18_w2v')